### 의류 종류 인식
의류의 분류별로 인식하게끔 하는 과정을 진행해보자.

### 데이터 수집
크롤링으로 하이버에서 종류별로 이미지를 가져온다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


# 옵션 정의
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

In [2]:
# 스크롤 다운 함수 정의

def scroll_down(body):
    for _ in range(1, 4):
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)

### Image Url 수집

In [7]:
from urllib.parse import quote
from selenium.webdriver.common.by import By

driver = webdriver.Chrome("./114/chromedriver.exe", options = options)

label_dict = {1 : "니트", 2 : "아노락", 3 : "후드"}

src_list = []
for value in label_dict.values():
    img_src = []
    for j in range(1, 4):
        url = f"https://www.musinsa.com/mz/brandsnap?swh=&stx={quote(value)}&_m=&gender=&mod=&bid=&p={j}"
        driver.get(url)
        driver.implicitly_wait(5)
        body = driver.find_element(By.CSS_SELECTOR, "body")
        
        for i in range(3):
            scroll_down(body)
        
        imgs = driver.find_elements(By.TAG_NAME, "img")
        for img in imgs:
            src = img.get_attribute("src")
            img_src.append(src)
            
    src_list.append(img_src)

driver.close()

### 수집된 URL로부터 이미지 다운로드

In [8]:
import requests
from urllib.request import urlretrieve

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}

label_dict = {1 : "knitwear", 2 : "anorak", 3 : "hoodie"}

i = 0
for idx, srcs in enumerate(src_list):
    for src in srcs:
        try:
            urlretrieve(src, f"./images/{i}_{label_dict[idx+1]}.jpeg")
        except:
            req = requests.get(src, headers = headers)
            with open(f"./images/{i}_{label_dict[idx+1]}.jpeg", "wb") as file:
                file.write(req.content)
        i += 1

In [20]:
import os, shutil

cnt_knit = []
cnt_anorak = []
cnt_hoodie = []

for filename in os.listdir("./images/"):
    if "knitwear" in filename:
        cnt_knit.append(filename)
    elif "anorak" in filename:
        cnt_anorak.append(filename)
    elif "hoodie" in filename:
        cnt_hoodie.append(filename)

In [17]:
print(len(os.listdir("./images/")))

500


In [16]:
for to_test in cnt_knit[:15]:
    shutil.move(f"./images/{to_test}", f"./data/test/images/{to_test}")
    
for to_test in cnt_anorak[:15]:
    shutil.move(f"./images/{to_test}", f"./data/test/images/{to_test}")
    
for to_test in cnt_hoodie[:10]:
    shutil.move(f"./images/{to_test}", f"./data/test/images/{to_test}")

In [21]:
for to_train in cnt_knit[:126]:
    shutil.move(f"./images/{to_train}", f"./data/train/images/{to_train}")
    
for to_train in cnt_anorak[:126]:
    shutil.move(f"./images/{to_train}", f"./data/train/images/{to_train}")
    
for to_train in cnt_hoodie[:126]:
    shutil.move(f"./images/{to_train}", f"./data/train/images/{to_train}")

In [23]:
for to_valid in cnt_knit[126:]:
    shutil.move(f"./images/{to_valid}", f"./data/validation/images/{to_valid}")
    
for to_valid in cnt_anorak[126:]:
    shutil.move(f"./images/{to_valid}", f"./data/validation/images/{to_valid}")
    
for to_valid in cnt_hoodie[126:]:
    shutil.move(f"./images/{to_valid}", f"./data/validation/images/{to_valid}")

In [ ]:
print(len(os.listdir("./train/images/")), len(os.listdir("./validation/images/")))

378 162


### 이미지 라벨링1 (1, 2중 택1)
labelImg를 활용한 라벨링

#### Download
1. https://github.com/tzutalin/labelImg 우측 하단의 릴리즈 클릭
2. 버전에 맞게 다운로드 후 cmd 내부에서 해당 폴더의 위치로 이동
3. 아래 순서대로 실행
```
>>> pip install PyQt5
>>> pip install lxml
>>> pyrcc5 -o libs/resources.py resources.qrc 
>>> pip3 install labelImg 
>>> python labelimg.py
```

> pyrcc5 -o libs/resources.py resources.qrc 의 경우 pyqt에 대한 환경변수 설정이 되어 있어야 함. <p>
> 환경변수 편집 ➡️ 시스템 변수 ➡️ 새로 만들기 ➡️ 변수이름 : pyrcc5, 변수값 : PyQt5가 설치된 경로

#### Usage
0. 설치된 폴더들을 살펴보면 data란 폴더가 있는데, 그 안에 있는 predefined_classes.txt파일을 눌러서 본인이 분류하고자 하는 클래스 이름으로 변경해야한다.
1. `python labelimg.py`이 실행되면 GUI 창 하나가 켜진다. 좌측 하단의 Yolo 버튼을 세번 눌러서 다시 Yolo로 맞춘다.
2. 편집할 이미지가 담긴 디렉토리를 불러온다.
3. w를 누르면 커서가 바뀌면서 어노테이션 시작. Save버튼 누르는거 잊지 않기...

In [32]:
%cd C:\Users\OWNER\Desktop\labeling\labelImg-master
!python labelimg.py

C:\Users\OWNER\Desktop\labeling\labelImg-master
[('knitwear', [(85, 73), (168, 73), (168, 158), (85, 158)], None, None, False), ('jeans', [(83, 149), (169, 149), (169, 340), (83, 340)], None, None, False)]
[('knitwear', [(85, 73), (168, 73), (168, 158), (85, 158)], None, None, False), ('jeans', [(83, 149), (169, 149), (169, 340), (83, 340)], None, None, False)]
[('knitwear', [(86, 64), (153, 64), (153, 161), (86, 161)], None, None, False), ('long pants', [(82, 152), (157, 152), (157, 360), (82, 360)], None, None, False)]
[('knitwear', [(67, 60), (172, 60), (172, 184), (67, 184)], None, None, False), ('long pants', [(85, 178), (165, 178), (165, 354), (85, 354)], None, None, False)]
[('knitwear', [(90, 64), (177, 64), (177, 168), (90, 168)], None, None, False), ('short pants', [(88, 158), (162, 158), (162, 239), (88, 239)], None, None, False)]
[('knitwear', [(75, 64), (171, 64), (171, 189), (75, 189)], None, None, False), ('long pants', [(91, 172), (159, 172), (159, 316), (91, 316)], Non

In [33]:
%cd D:\Intel\

D:\Intel


### 이미지 라벨링2
roboflow를 활용한 라벨링

1. Roboflow 회원가입(WorkSpace생성)
2. 프로젝트 생성
    + 2-1. Project Type : Object detection
    + 2-2. What Are You Detecting? : 분류하고자 하는 것들의 대분류(여기서는 옷을 의미하는 cloth)
    + 2-3. Project Name : 프로젝트가 생성될 이름(저는 cloth detection으로 생성했습니다.)
    + 2-4. License : CC BY 4.0
3. 이미지 폴더 업로드 후 우측 상단 `Save and Continue` 클릭 이후 오른쪽에서 탭 하나가 튀어나올텐데 하단에 `Assing Image`클릭
4. 우측 상단의 `Start Annotating` 누르며 어노테이션 시작(첫 이미지 어노테이션은 바로 되는데 두번째부터는 ctrl을 누르고 진행해야함)
5. 어노테이션 완료 후 Generate탭 클릭
6. 차례대로 진행
7. 우측 상단의 Export Dataset 클릭(Format 형태 : YoloV5 Pytorch, 다운로드 형태 선택)
8. 다운된 데이터를 기반으로 훈련

### clone yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


### Yaml 파일 수정

In [38]:
import yaml

with open('./YOLO/yolov5/data/coco.yaml', "r") as f:
    data = yaml.safe_load(f)
    
data["path"] = r"D:\Intel\data\\"
data["train"] = "train"
data["test"] = "test"
data["val"] = "validation"

data["nc"] = 7
data["names"] = ["knitwear", "anorak", "hoodie", "jeans", "long pants", "short pants", "skirt"]

with open("./YOLO/yolov5/data/coco.yaml", "w") as file:
    yaml.dump(data, file)

In [2]:
%pwd

'd:\\Intel'

### 훈련
clone한 파일들 중 `train.py`를 이용해서 훈련을 진행한다. argparse로 작성되어 여러 옵션들을 함께 지정해주어야 한다.
- --batch : batch_size
- --epochs : 훈련 횟수
- --data : data정보를 담고있는 yaml 파일
- --weights : pretrained 된 가중치를 적용할 때 사용. 지정하지 않으면 랜덤한 가중치와 편향값들로 학습 진행
- --cfg : 모델을 담고있는 yaml파일. s ➡️ n ➡️ m ➡️ l ➡️ x 순으로 복잡도가 다르다. 

In [1]:
!python ./YOLO/yolov5/train.py --batch 4 --epochs 150 --data ./YOLO/yolov5/data/coco.yaml  --weights ./YOLO/yolov5/yolov5m.pt --cfg ./YOLO/yolov5/models/yolov5m.yaml

^C


train: weights=./YOLO/yolov5/yolov5m.pt, cfg=./YOLO/yolov5/models/yolov5m.yaml, data=./YOLO/yolov5/data/coco.yaml, hyp=YOLO\yolov5\data\hyps\hyp.scratch-low.yaml, epochs=150, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=0, project=YOLO\yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
From https://github.com/ultralytics/yolov5
   0897415..df48c20  master     -> origin/master
github:  YOLOv5 is out of date by 4 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-203-g0897415 Python-3.9.12 torch-1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB

### 검출
위에서 훈련한 나만의 모델을 기반으로 새로운 이미지들에 대한 객체 탐지 결과를 `detect.py`파일로 받아본다. 마찬가지로 argparse로 작성되어있다.
- --source : 탐지할 이미지 경로
- --weights : 나의 학습 모델의 가중치 경로
- --conf : 객체 탐지 신뢰도. 해당 신뢰도 이상의 객체들만 탐지한다. 기본값은 0.4이며 높을 수록 탐지되는 객체의 수도 적어진다.
- --name : 탐지된 파일들을 저장할 경로

In [1]:
!python ./YOLO/yolov5/detect.py --source "C:\Users\OWNER\Desktop\cloth_detection_test" --weights "./YOLO/yolov5/runs/train/exp9/weights/best.pt" --conf 0.65 --name "C:\Users\OWNER\Desktop\cloth_detection_test\results"

detect: weights=['./YOLO/yolov5/runs/train/exp9/weights/best.pt'], source=C:\Users\OWNER\Desktop\cloth_detection_test, data=YOLO\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.65, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=YOLO\yolov5\runs\detect, name=C:\Users\OWNER\Desktop\cloth_detection_test\results, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-203-g0897415 Python-3.9.12 torch-1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20877180 parameters, 0 gradients, 47.9 GFLOPs
image 1/9 C:\Users\OWNER\Desktop\cloth_detection_test\cloth_test1.jpg: 640x640 1 hoodie, 16.0ms
image 2/9 C:\Users\OWNER\Desktop\cloth_detection_test\cloth_test2.jpg: 640x512 1 hoodie, 14.7ms
image 3/9 C:\User

In [1]:
import cv2
import torch
from imutils.video import WebcamVideoStream

model = torch.hub.load('./YOLO/yolov5', 'custom', path = "./YOLO/yolov5/runs/train/exp9/weights/best.pt", source = "local")
model.conf = 0.3

esp_ip = "http://192.168.0.39"
host = "{}:4747/video".format(esp_ip)      
cam = WebcamVideoStream(src=host).start()    

while True:
    frame = cam.read()
    results = model(frame)
    results.render()
    cv2.imshow('Modi Camera Module', frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

YOLOv5  v7.0-203-g0897415 Python-3.9.12 torch-1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20877180 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


### 결과
학습된 모델의 상태가 좋지 않다. 원인을 따져보자면 아래와 같을 것이다.
- 클래스 간의 상관관계가 너무 높음
    + 현재 구성한 클래스는 [반팔, 긴팔, 맨투맨, 후드, 니트]로, 니트의 경우 반팔 니트, 긴팔 니트가 존재하는 등의 상관관계가 높은 것이 아닌가 하는 생각이 든다
- 노이즈
    + Roboflow에서 제공하는 이미지 증강이 오히려 역효과를 낸 건 아닐까 하는 마음.. 
    + https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=4u_olion&logNo=221437862590 참고

### 보완
기존에 roboflow로 노이즈도 섞고 해서 나온 약 900장 가량의 데이터로 학습 시킨 데 반해, 이번에는 labelImg를 통해 어노테이션만 진행하되 기존의 이미지 형식과 다른 마켓 진열 사진이 아닌 사람이 입고 있는 사진에 별도의 클래스들을 추가했다.

각 데이터셋의 개수는 아래와 같다.
- train : 315장
- test : 81장
- validation : 122장

아래는 바뀐 클래스 명이다.

|변경 전|변경 후|
|---|---|
|['long', 'short', 'man2man', 'knitwear', 'hoodie']|["knitwear", "anorak", "hoodie", "jeans", "long pants", "short pants", "skirt"]|

단순히 상의에 국한되지 않고 하의 정보까지 추가하여 어노테이션을 진행하고, 별도의 이미지 증강 처리 없이 학습을 진행했다.

총 두 번의 시도가 있었는데, 변경점들만 짧게 살펴본다.

1회차

|변경점|내용|
|---|---|
|epochs|횟수 감소(200 ➡️ 100)|
|valid, test|모두 train set으로 진행|

2회차

|변경점|내용|
|---|---|
|epochs|횟수 증가(100 ➡️ 150)|
|weights|yolov5n ➡️ yolov5m으로 변경|
|valid, test|별도의 valid, test셋으로 진행|